# Assignment Part 1 Brief 

## Deadline: Friday, December 3, 2021 at 15:00 hrs

## Number of marks available for Part 1: 25

## Scope: Sessions 1 to 5

### Please READ the whole assignment first, before starting to work on it.

### How and what to submit

A. A **Jupyter Notebook** with the code in all the cells executed and outputs displayed.

B. Name your Notebook **COM4509-6509_Assignment_Part1_Username_XXXXXX.ipynb** where XXXXXX is your username such as such as abc18de.  

C. Upload a .zip file to Blackboard before the deadline that contains the Jupyter Notebook in B and any other files requested for the solution of Part 2 of the Assignment (Dr Lu will be in charge of releasing this part). 

D. **NO DATA UPLOAD**: Please do not upload the data files used in this Notebook. We have a copy already. 


### Assessment Criteria 

* Being able to use numpy and pandas to preprocess a dataset.

* Being able to use numpy to build a machine learning pipeline for supervised learning. 

* Being able to follow the steps involved in an end-to-end project in machine learning.

* Being able to use scikit-learn to design a machine learning model pipeline


### Late submissions

We follow Department's guidelines about late submissions, i.e., a deduction of 5% of the mark each working day the work is late after the deadline. NO late submission will be marked one week after the deadline because we will release a solution by then. Please read [this link](https://sites.google.com/sheffield.ac.uk/comughandbook/general-information/assessment/late-submission) if you are taking COM4509 or read [this link](https://sites.google.com/sheffield.ac.uk/compgtstudenthandbook/menu/assessment/late-submission) if you are taking COM6509. 

### Use of unfair means 

**"Any form of unfair means is treated as a serious academic offence and action may be taken under the Discipline Regulations."** (from the students Handbook). Please carefully read [this link](https://sites.google.com/sheffield.ac.uk/comughandbook/general-information/assessment/unfair-means) on what constitutes Unfair Means if not sure, for COM4509. If you are taking COM6509, please read [this link](https://sites.google.com/sheffield.ac.uk/compgtstudenthandbook/menu/referencing-unfair-means) if you are not sure what is Unfair means. If you still have questions, please ask your Personal tutor or the Lecturers. 

# A dataset of air quality

The dataset you will use in this assignment comes from a popular machine learning repository that hosts open source datasets for educational and research purposes, the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php). We are going to use regularised ridge regression and random forests for predicting air quality. The description of the dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/Air+Quality).

In [174]:
import urllib.request
doq = "https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip"
pat_sav = "./AirQualityUCI.zip"
result = urllib.request.urlretrieve(doq, pat_sav)
print(result)
#urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip")

('./AirQualityUCI.zip', <http.client.HTTPMessage object at 0x7fb878a66310>)


In [175]:
import zipfile
zip = zipfile.ZipFile('./AirQualityUCI.zip', 'r')
for name in zip.namelist():
    zip.extract(name, '.')

In [176]:
# The .csv version of the file has some typing issues, so we use the excel version
import pandas as pd 
air_quality_full = pd.read_excel('./AirQualityUCI.xlsx', usecols=range(2,15))

We can see some of the rows in the dataset 

In [177]:
air_quality_full.sample(5)

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
3179,-200.0,1010.75,-200,5.379850,782.25,75.0,827.75,73.0,1562.00,1115.5,23.25,50.100000,1.410012
7294,4.3,1532.25,-200,18.886468,1265.00,643.0,481.75,194.0,1577.50,1776.5,13.25,59.000000,0.894486
7448,2.0,1051.00,-200,6.182719,820.25,284.8,778.25,121.7,1087.25,1009.0,4.75,79.425001,0.687233
4453,-200.0,1024.25,-200,4.872898,757.00,-200.0,923.00,-200.0,1422.50,907.5,20.60,59.825000,1.435008
8110,7.1,-200.00,-200,-200.000000,-200.00,1218.0,-200.00,339.7,-200.00,-200.0,-200.00,-200.000000,-200.000000


The target variable corresponds to the CO(GT) variable of the first column. The following columns correspond to the variables in the feature vectors, *e.g.*, PT08.S1(CO) is $x_1$ up until AH which is $x_D$. The original dataset also has a date and a time columns that we are not going to use in this assignment.

### Removing instances 

The dataset has missing values tagged with a -200 value. To simplify the design of the machine learning models in this assignment, we perform the following two operations to the dataset right from the beginning:

* we will remove the rows for which the target variable has missing values. We are doing supervised learning so we need all our data observations to have known target values.

* we will remove features with more than 20% of missing values. 

The code below performs both operations

In [178]:
# We first remove the rows for which there are missing values in the target feature
air_quality = air_quality_full.loc[air_quality_full.iloc[:, 0]!=-200, :]

In [179]:
# We now remove the columns (features) for which there are more that 20% of missing values
import numpy as np
ndata, ncols = np.shape(air_quality) # number of data observations and number of columns in the dataframe
pmissing = np.empty(ncols)         # An empty vector that will keep the percentage of missing values per feature
for i in range(ncols):
    pmissing[i] = (air_quality.iloc[:, i]==-200).sum()/ndata # Computes the percentage of missing values per column
air_quality = air_quality.loc[:, pmissing < 0.2]

### Splitting the dataset 

Before designing any machine learning model, we need to set aside the test data. We will use the remaining training data for fitting the model. *It is important to remember that the test data has to be set aside before preprocessing*. 

Any preprocessing that you do has to be done only on the training data and several key statistics need to be saved for the test stage.  Separating the dataset into training and test before any preprocessing has happened help us to recreate the real world scenario where we will deploy our system and for which the data will come without any preprocessing.

Furthermore, we are going to use *hold-out validation* for validating our predictive model so we need to further separate the training data into a training set and a validation set.

We split the dataset into a training set, a validation set and a test set. The training set will have 70% of the total observations, the validation set 15% and the test set, the remaining 15%. For making the random selections of the training and validation sets **make sure that you use a random seed that corresponds to the last five digits of your student UCard**. In the code below, I have used 55555 as an example of my random seed.

In [180]:
np.random.seed(63196)                 # Make sure you use the last five digits of your student UCard as your seed
index = np.random.permutation(ndata)  # We permute the indexes 
N = np.int64(np.round(0.70*ndata))    # We compute N, the number of training instances
Nval = np.int64(np.round(0.15*ndata)) # We compute Nval, the number of validation instances   
Ntest = ndata - N - Nval              # We compute Ntest, the number of test instances
data_training_unproc = air_quality.iloc[index[0:N], :].copy() # Select the training data
data_val_unproc = air_quality.iloc[index[N:N+Nval], :].copy() # Select the validation data
data_test_unproc = air_quality.iloc[index[N+Nval:ndata], :].copy() # Select the test data
data_training_unproc

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
692,2.8,1144.00,13.903863,1113.50,181.0,844.75,128.0,1550.75,1009.50,20.650,22.575000,0.543151
5978,1.1,811.50,2.995843,651.75,210.0,1585.50,106.0,963.75,554.00,9.150,49.625000,0.576455
3015,3.7,1211.00,19.761560,1289.75,311.0,605.75,156.0,1913.00,1642.75,24.000,36.375000,1.070554
5854,1.5,909.00,7.522327,879.25,-200.0,873.75,-200.0,1083.00,846.75,13.825,40.725000,0.640528
9078,1.9,1250.75,9.883450,973.25,313.7,558.50,143.6,1276.50,1481.50,15.900,47.025000,0.843446
...,...,...,...,...,...,...,...,...,...,...,...,...
2085,1.3,925.75,7.264107,868.25,80.0,1047.75,75.0,1599.25,687.25,27.600,35.100000,1.276385
8374,2.5,1174.00,10.866450,1009.50,359.9,612.75,218.2,1304.50,1049.75,7.400,69.275000,0.716549
296,0.7,940.50,2.525727,621.25,51.0,1298.25,70.0,1329.75,714.25,12.950,56.774999,0.844362
4050,1.6,-200.00,-200.000000,-200.00,90.0,-200.00,77.0,-200.00,-200.00,-200.000,-200.000000,-200.000000


The assigment is divided into two sections. In the **first section**, you will design a regularised ridge regression model trained with stochastic gradient descent. You will write all the code from scratch. You should not use any library that already implements any of the routines considered in this section, for example, scikit-learn. In the **second section**, you will design a random forests model and you are allowed to use scikit-learn in this section.

When writing your code, you will find out that there are operations that are repeated at least twice. We will assign marks for use of Python functions and for commenting your code. The marks will be assigned as:

* Did you include Python functions to solve the question and avoid repeating code? (**1 mark**)
* Did you comment your code to make it readable to others? (**1 mark**)

# 1. Using regularised ridge regression to predict air quality (10 marks)

**DO NOT USE scikit-learn or any other machine learning library for the questions on this section. You are meant to write Python code from scratch. You can use Pandas and Numpy. Using scikit-learn or any other machine learning library for the questions in this section will give ZERO marks. No excuse will be accepted.**

Regularisation is a technique commonly used in Machine Learning to prevent overfitting. It consists on adding terms to the objective function such that the optimisation procedure avoids solutions that just learn the training data. Popular techniques for regularisation in Supervised Learning include [Lasso Regression](https://en.wikipedia.org/wiki/Lasso_(statistics)), [Ridge Regression](https://en.wikipedia.org/wiki/Tikhonov_regularization) and the [Elastic Net](https://en.wikipedia.org/wiki/Elastic_net_regularization). 

In this part of the Assignment, you will be looking at Ridge Regression and implementing equations to optimise the objective function using the update rules for stochastic gradient descent. You will use those update rules for making predictions on the Air Quality dataset.

## 1.1 Ridge Regression

Let us start with a data set for training $\mathcal{D} = \{\mathbf{y}, \mathbf{X}\}$, where the vector $\mathbf{y}=[y_1, \cdots, y_N]^{\top}$ and $\mathbf{X}$ is the design matrix from Lab 4, this is, 

\begin{align*}
    \mathbf{X} = 
                \begin{bmatrix}
                        1 & x_{1,1} & \cdots & x_{1, D}\\
                        1 & x_{2,1} & \cdots & x_{2, D}\\
                   \vdots &  \vdots\\
                        1 & x_{N,1} & \cdots & x_{N, D}
                \end{bmatrix}
               = 
               \begin{bmatrix}
                      \mathbf{x}_1^{\top}\\
                       \mathbf{x}_2^{\top}\\
                          \vdots\\
                        \mathbf{x}_N^{\top}
                \end{bmatrix}.
\end{align*}

Our predictive model is going to be a linear model

$$ f(\mathbf{x}_i) = \mathbf{w}^{\top}\mathbf{x}_i,$$

where $\mathbf{w} = [w_0\; w_1\; \cdots \; w_D]^{\top}$.

The **objective function** we are going to use has the following form

$$ E(\mathbf{w}, \lambda) = \frac{1}{N}\sum_{n=1}^N (y_n - f(\mathbf{x}_n))^2 + \frac{\lambda}{2}\sum_{j=0}^D w_j^2,$$

where $\lambda>0$ is known as the *regularisation* parameter.

This objective function was studied in Lecture 4. 

The first term on the rhs is what we call the "fitting" term whereas the second term in the expression is the regularisation term. Given $\lambda$, the two terms in the expression have different purposes. The first term is looking for a value of $\mathbf{w}$ that leads the squared-errors to zero. While doing this, $\mathbf{w}$ can take any value and lead to a solution that it is only good for the training data but perhaps not for the test data. The second term is regularising the behavior of the first term by driving the $\mathbf{w}$ towards zero. By doing this, it restricts the possible set of values that $\mathbf{w}$ might take according to the first term. The value that we use for $\lambda$ will allow a compromise between a value of $\mathbf{w}$ that exactly fits the data (first term) or a value of $\mathbf{w}$ that does not grow too much (second term).

This type of regularisation has different names: ridge regression, Tikhonov regularisation or $\ell_2$ norm regularisation. 

## 1.2 Optimising the objective function with respect to $\mathbf{w}$

There are two ways we can optimise the objective function with respect to $\mathbf{w}$. The first one leads to a closed form expression for $\mathbf{w}$ and the second one using an iterative optimisation procedure that updates the value of $\mathbf{w}$ at each iteration by using the gradient of the objective function with respect to $\mathbf{w}$,
$$
\mathbf{w}_{\text{new}} = \mathbf{w}_{\text{old}} - \eta \frac{d E(\mathbf{w}, \lambda)}{d\mathbf{w}},
$$
where $\eta$ is the *learning rate* parameter and $\frac{d E(\mathbf{w}, \lambda)}{d\mathbf{w}}$ is the gradient of the objective function.

It can be shown (this is a question in the Exercise Sheet 4) that a closed-form expression for the optimal $\mathbf{w}_*$ is given as

\begin{align*}            
            \mathbf{w}_*& = \left(\mathbf{X}^{\top}\mathbf{X} + \frac{\lambda N}   
                                     {2}\mathbf{I}\right)^{-1}\mathbf{X}^{\top}\mathbf{y}.
\end{align*}

Alternatively, we can find an update equation for $\mathbf{w}_{\text{new}}$ using gradient descent leading to:

\begin{align*}
   \mathbf{w}_{\text{new}} & = \mathbf{w}_{\text{old}} - \eta \frac{d E(\mathbf{w}, \lambda)}
                              {d\mathbf{w}},\\
                           & = \mathbf{w}_{\text{old}} +  \frac{2\eta}{N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n  
                       - \eta\lambda\mathbf{w}_{\text{old}}\\
                           & = (1 - \eta\lambda)\mathbf{w}_{\text{old}} + \frac{2\eta}
                               {N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n
\end{align*}

## 1.3 Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 1.a: Missing values and standardisation (2 marks)

* For all the other features with missing values, use the mean value of the non-missing values for that feature to perform imputation. Save these mean values, you will need them when performing the validation stage (**1 mark**).

* Once you have imputed the missing data, we need to standardise the input vectors. Standardise the training data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Keep the mean values and standard deviations, you will need them at validation time (**1 mark**).

#### Question 1.a Answer

In [354]:
data, cols = np.shape(data_training_unproc)  # Calculating the number of rows and columns in the training data
mean_nmissing = np.empty(cols)      # To store mean of non-missing values
mean_after_impute = np.empty(cols)  # To store the mean after imputation
stddev_after_impute = np.empty(cols)  # To store standard deviation after imputation
data_training_proc = data_training_unproc

def train_impute(col):
    mean_nmissing[col] = data_training_unproc[data_training_unproc.iloc[:,col] != -200].iloc[:,col].mean()  # Calculate and store mean of non missing values in each column
    data_training_proc.iloc[:,col].replace(-200 , mean_nmissing[col], inplace = True) 
    
def train_standardise(col):
    mean_after_impute[col] = data_training_proc.iloc[:,col].mean()  # Calculate mean of each feature after imputation
    stddev_after_impute[col] = data_training_proc.iloc[:,col].std()
    data_training_proc.iloc[:,col] =(data_training_proc.iloc[:,col] - mean_after_impute[col])/stddev_after_impute[col]
    
for col in range(1,cols):
    train_impute(col)    # Step 1: Imputation
    train_standardise(col)   # Step 2: Standardisation 
    
# dtype = np.dtype("float64")

## 1.4 Training and validation stages

We have now curated our training data by removing data observations and features with a large amount of missing values. We have also normalised the feature vectors. We are now in a good position to work on developing the prediction model and validating it. We will use gradient descent for iterative optimisation. 

We first organise the dataframe into the vector of targets $\mathbf{y}$, call it `yTrain`, and the design matrix $\mathbf{X}$, call it `XTrain`.

In [355]:
# Function to create design matrix and target vector
def split_x_and_y(rows, cols, dataset):
    x = np.array(dataset.iloc[:, 1:cols]).reshape(-1, cols-1)
    XTrain = np.hstack((np.ones((rows,1)), x))   # Creating the design matrix -- Referenced from Lab4
    yTrain = np.array(dataset.iloc[:, 0].values).reshape(-1,1)  # Creating target vector
    return (XTrain, yTrain)

In [356]:
XTrain, yTrain = split_x_and_y(data, cols, data_training_proc)
print(XTrain.shape, yTrain.shape)

(5372, 12) (5372, 1)


### Question 1.b: finding the optimal $\mathbf{w}$ with stochastic gradient descent (3 marks)

Use gradient descent to iteratively compute the value of $\mathbf{w}_{\text{new}}$. Instead of using all the training set in `XTrain` and `yTrain` to compute the gradient, use a subset of $S$ instances in `XTrain` and `yTrain`. This is sometimes called *minibatch gradient descent* where $S$ is the size of the minibacth. When using gradient descent with minibatches, you need to find the best values for three parameters: $\eta$, the learning rate, $S$, the number of datapoints in the minibatch and $\gamma$, the regularisation parameter.

* In this question we will use the validation data. So before proceeding to the next steps, make sure that you:  replace the missing values on each feature variables with the mean value you computed with the training data; standardise the validation data using the means and standard deviations computed from the training data (**1 mark**).
    
* Create a grid of values for the parameters $\gamma$ and $\eta$ using `np.logspace` and a grid of values for $S$ using `np.linspace`. Because you need to find three parameters, start with five values for each parameter in the grid and see if you can increase it. Make sure you understand what is the meaning of `np.logspace` and `np.linspace`. Notice that you can use negative values for `start` in `np.logspace` (**1 mark**).

* For each value that you have of $\gamma$, $\eta$ and $S$ from the previous step, use the training set to compute $\mathbf{w}$ using minibatch gradient descent and then measure the RMSE over the validation data. For the validation data, make sure you preprocess it before applying the prediction model over it. For the minibatch gradient descent choose to stop the iterative procedure after $200$ iterations (**1 mark**).

* Choose the values of $\gamma$, $\eta$ and $S$ that lead to the lowest RMSE and save them. You will use them at the test stage.

#### Question 1.b Answer

In [357]:
# Preprocessing functions

def impute(col, dataset):
    dataset.iloc[:,col].replace(-200 , mean_nmissing[col], inplace = True)  
    
def standardise(col, dataset):
    dataset.iloc[:,col] = (dataset.iloc[:,col] - mean_after_impute[col])/stddev_after_impute[col]
    

In [358]:
vdata, vcols = np.shape(data_val_unproc)  # Calculating the number of rows and columns in the validation data
data_val_proc = data_val_unproc

for col in range(1,vcols):
    impute(col, data_val_proc)    # Step 1: Imputation
    standardise(col, data_val_proc)   # Step 2: Standardisation

In [520]:
# Split data into design matrix and feature vector
XVal, yVal = split_x_and_y(vdata, vcols, data_val_proc)
print(XVal.shape, yVal.shape)

(1151, 12) (1151, 1)


In [527]:
batch_size = np.linspace(start = 100, stop = 800, num = 5, dtype = np.int64 )
learning_rate = np.logspace(start = -4, stop = 0, num = 5)
regularisation = np.logspace(start = -6, stop = -1, num = 5)

w_old = np.array(np.ones(XVal.shape[1])).reshape(-1,1)
pd.DataFrame(w_old)
w_new = np.empty(w_old.shape)
error_rmse = 99999999

In [528]:
#Minibatch Gradient Descent
b_S = 0             # To store best batch_size
b_ita = 0.0         # To store best learning_rate
b_gamma = 0.0       # To store best regularisation
loss = 0

def get_mini_batch(X, y, batch_Size):         # Reference: GeeksforGeeks.com
    mini_batches = []
    data = np.hstack((X, y))
    np.random.shuffle(data)
    num_batches = data.shape[0] // batch_Size  # Find the number of full size batches    
    
    for i in range(num_batches):
        mini_batch = data[i * batch_Size:(i + 1)*batch_Size, :]
        X_mini = mini_batch[:, :-1]
        Y_mini = mini_batch[:, -1].reshape((-1, 1))
        mini_batches.append((X_mini, Y_mini))
    
    if data.shape[0] % batch_Size != 0:           # Put the remaining elements(if any) in the last batch
        mini_batch = data[(i+1) * batch_Size:data.shape[0]]
        X_mini = mini_batch[:, :-1]
        Y_mini = mini_batch[:, -1].reshape((-1, 1))
        mini_batches.append((X_mini, Y_mini))
    return mini_batches


def get_best_param(X, Y, gamma, ita, S, w_old):
    for epoch in range(200):
            mini_batches = get_mini_batch(X,Y,S)
            
            for mini_batch in mini_batches:
                X_mini, y_mini = mini_batch
                loss_term = (1- ita*gamma) * w_old
                temp = np.dot(((np.dot(X_mini, w_old)).T) , X_mini)
                reg_term = 2*ita/S + (np.dot(y_mini.T, X_mini) - temp)
                w_new = loss_term + reg_term.T
    return w_new

def get_RMSE(w, x, y):
    y_pred = np.dot(x, w)
    loss = (y - y_pred)
    loss = np.array(loss, dtype = float)
    rmse = np.sqrt(np.mean(np.square(loss)))
    return rmse


# Find best parameters
for S in batch_size:
    for ita in learning_rate:
        for gamma in regularisation:
            w_new = get_best_param(XTrain, yTrain, gamma, ita, S, w_old)
            w_old = w_new
    loss = get_RMSE(w_new, XVal, yVal )
    if loss < error_rmse:
        error_rmse = loss
        b_S = S
        b_ita = ita
        b_gamma = gamma
    print("Finished Combination")

Finished Combination
Finished Combination
Finished Combination
Finished Combination
Finished Combination


In [529]:
print("Best Parameters are:", 
      "\nBatch Size:", b_S, 
     "\nLearning Rate:", b_ita,
     "\nRegularisation:", b_gamma)

Best Parameters are: 
Batch Size: 100 
Learning Rate: 1.0 
Regularisation: 0.1


## 1.5 Test stage 

We now know which one is the best model, according to the validation data. We will now put together the training data and the validation data, perform the preprocessing as we did before, this is, treat the missing values and standardise the inputs. We will train the model again using the minibatch stochastic gradient descent and finally compute the RMSE over the test data.


### Question 1.c: combine the original training and original validation data and perform the preprocessing again to this new data (2 marks)

Put together the original training and validation dataset and perform the same preprocessing steps than before, these are: 

* for each feature, impute the missing values with the mean values of the non-missing values (**1 mark**) 

* stardardise the new training set (**1 mark**).

#### Question 1.c Answer

In [530]:
# Merging the validation and training data
new_data_unproc = data_training_unproc + data_val_unproc
new_data_unproc.shape

(6523, 12)

In [531]:
data, cols = np.shape(new_data_unproc)  # Calculating the number of rows and columns in the validation data
new_data_proc = new_data_unproc

for col in range(1, cols):
    impute(col, new_data_proc)    # Step 1: Imputation
    standardise(col, new_data_proc)   # Step 2: Standardisation

In [532]:
# Split data into design matrix and feature vector
XTrainNew, yTrainNew = split_x_and_y(data, cols, new_data_proc)
print(XTrainNew.shape, yTrainNew.shape)

(6523, 12) (6523, 1)


### Question 1.d: Preprocess the test data, train the model and predict over the test data (3 marks)

Preprocess the test data and train a new model using the new training set. Finally, report the RMSE over the test set:

* Preprocess the test data by imputing the missing data and standardising it (**1 mark**). 

* Use the best values of $\gamma$, $\eta$ and $S$ found in the validation set and train a new regularised linear model with stochastic gradient descent (**1 mark**).

* Report the RMSE over the test data (**1 mark**).

#### Question 1.d Answer

In [533]:
data, cols = np.shape(data_test_unproc)  # Calculating the number of rows and columns in the validation data
data_test_proc = data_test_unproc

for col in range(1, cols):
    impute(col, data_test_proc)    # Step 1: Imputation
    standardise(col, data_test_proc)   # Step 2: Standardisation

In [534]:
# Split data into design matrix and feature vector
XTest, yTest = split_x_and_y(data, cols, data_test_proc)
print(XTest.shape, yTest.shape)

(1151, 12) (1151, 1)


In [537]:
w_old = np.array(np.ones(XTrainNew.shape[1])).reshape(-1,1)

# Train model on new dataset
w_new = get_best_param(XTrainNew, yTrainNew, b_gamma, b_ita, b_S, w_old)

# Calculate the rmse loss on test dataset
loss = get_RMSE(w_new, XTest, yTest)
print("Loss:", loss)  

Loss: nan


# 2. Random forests (13 marks)

**USE scikit-learn for the questions on this section.**

In section 1, you used a regularised ridge regression model trained with SGD to create a linear predictive model. In this part of the assignment, you will use **scikit-learn** to train a random forest for regression over the air quality dataset.

## 2.1 Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 2.a: Pipeline for missing values and standardisation (3 marks)

* Employ the `SimpleImputer` method in Scikit-learn to impute the missing values in each column using the mean value of the non-missing values, instead (**1 mark**).

* Standardise the data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Employ the `StandardScaler` method (**1 mark**).

* Create a `Pipeline` that you can use to preprocess the data by filling missing values and then standardising the features (**1 mark**).

#### Question 2.a Answer

In [516]:
# This solution uses references from the Lab 3 of the COM 6509 module

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

data, cols = np.shape(data_training_unproc) # Used as in question one
data_training_proc = data_training_unproc
imputer = SimpleImputer(missing_values = -200, strategy = 'mean')
scaler = StandardScaler()

for col in range(1,cols):
    data_training_proc.iloc[:,col] = imputer.fit_transform(data_training_proc.iloc[:,col].values.reshape(-1,1))[:,0]  # Referenced from dzone.com
    data_training_proc.iloc[:,col] = scaler.fit_transform(data_training_proc.iloc[:,col].values.reshape(-1,1))[:,0]   # Standardise the data in each column


# Creating machine learning pipeline
model_pipeline = Pipeline(steps = [('imputation', SimpleImputer(missing_values = -200, strategy = 'mean')),
                                   ('standardisation', StandardScaler())])

pre_processor = ColumnTransformer([('preprocess_features', model_pipeline, data_training_unproc.iloc[:,1:cols].columns)], remainder = 'passthrough')

### Question 2.b: Use the Pipeline to fit the training data and transform the validation data (2 marks)

In the previous question, you created a `Pipeline` for applying a `SimpleImputer` and a `StandardScaler`. Use the Pipeline to fit the training data (**1 mark**) and transform the validation data (**1 mark**).

#### Question 2.b Answer

In [517]:
data_val_proc = data_val_unproc
pre_processor.fit(data_training_unproc) # Using the pipeline created above to preprocess the data

col_names = data_training_proc.columns
feature_cols = list(data_training_unproc.iloc[:,1:cols].columns)
feature_cols.append(col_names[0])

data_val_proc = pd.DataFrame(pre_processor.transform(data_val_proc), columns = feature_cols)
data_training_proc

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
692,2.8,1.412844e-01,4.811520e-01,6.257378e-01,-3.554296e-01,8.015665e-02,2.670783e-01,2.965211e-01,-9.289723e-02,3.266780e-01,-1.539443e+00,-1.137532e+00
5978,1.1,-1.409463e+00,-1.003485e+00,-1.146632e+00,-2.168347e-01,3.029970e+00,-2.029588e-01,-1.399336e+00,-1.243863e+00,-9.906805e-01,3.998952e-02,-1.052409e+00
3015,3.7,4.537657e-01,1.278415e+00,1.302251e+00,2.658576e-01,-8.715887e-01,8.653072e-01,1.343070e+00,1.507211e+00,7.104304e-01,-7.336696e-01,2.104891e-01
5854,1.5,-9.547323e-01,-3.874078e-01,-2.734016e-01,1.346565e-15,1.956404e-01,1.121838e-15,-1.054820e+00,-5.041370e-01,-4.551456e-01,-4.796758e-01,-8.886393e-01
9078,1.9,6.391558e-01,-6.604688e-02,8.740568e-02,2.787613e-01,-1.059748e+00,6.003773e-01,-4.957937e-01,1.099761e+00,-2.174483e-01,-1.118229e-01,-3.699888e-01
...,...,...,...,...,...,...,...,...,...,...,...,...
2085,1.3,-8.766120e-01,-4.225528e-01,-3.156237e-01,-8.381220e-01,8.885429e-01,-8.652837e-01,4.366388e-01,-9.071647e-01,1.122821e+00,-8.081160e-01,7.365863e-01
8374,2.5,2.812014e-01,6.774456e-02,2.265468e-01,4.995572e-01,-8.437133e-01,2.194230e+00,-4.149010e-01,8.807243e-03,-1.191148e+00,1.187341e+00,-6.943343e-01
296,0.7,-8.078195e-01,-1.067471e+00,-1.263702e+00,-9.767168e-01,1.886083e+00,-9.721103e-01,-3.419532e-01,-8.389406e-01,-5.553794e-01,4.574734e-01,-3.676498e-01
4050,1.6,6.082907e-17,5.197305e-15,-4.253452e-16,-7.903306e-01,-1.235586e-16,-8.225531e-01,-3.087437e-16,1.187077e-15,1.216030e-15,-8.874879e-16,-6.047863e-15


## 2.2 Random forest to predict air quality 

We now use random forests to predict air quality. Remember that the tree ensemble in random forests is built by training individual regression trees on different subsets of the training data and using a subset of the available features. For regression, the prediction is the average of the individual predictions of each tree. Some of the parameters required in the Random Forest implementation in Scikit-learn include:

Some of the additional parameters required in the Random Forest implementation in Scikit-learn include

> **n_estimators** the total number of trees to train<p>
**max_features** number of features to use as candidates for splitting at each tree node. <p>
    **boostrap**: Whether bootstrap samples are used when building trees. If False, the whole dataset is used to build each tree.<p>
   **max_samples**: If bootstrap is True, the number of samples to draw from X to train each base estimator.

### Question 2.c: train a random forest (4 marks)

In this question, you will train a random forest for predicting over the validation data. Use cross-validation over the validation data to select the best set of paramaters for the random forest regressor. Parameters to include in your exploration are **n_estimators**, **max_features** and **max_samples**. Use `np.linspace` or `np.logspace` to define ranges of values to explore for each parameter and create a grid to be assessed over the validation data. Make sure you use the same validation data that was given to you.

* Use `PredefinedSplit` to tell the cross validator which instances to use for training and which ones for validation (**1 mark**).

* Create a grid of values to explore that include a range of at least five values for each parameter **n_estimators**, **max_features** and **max_samples** (**1 mark**). 

* Train a random forest for regression model that uses the grid of parameters you created before. Use `GridSearchCV` to find the best set of parameters by performing cross-validation over the predefined split. (**1 mark**).

* Print the best values in your grid for **n_estimators**, **max_features** and **max_samples** (**1 mark**).

#### Question 2.c Answer

In [518]:
from sklearn.model_selection import PredefinedSplit, GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor

data_val_features = data_val_proc.iloc[:,0:cols-1]
data_val_target = data_val_proc.iloc[:,cols-1]


# Preparing the data
val_split_train, val_split_test = train_test_split(data_val_proc, test_size=0.20, random_state=42)  # Referenced from lab 3
train_features = val_split_train.iloc[:,0:cols-1]
train_target = val_split_train.iloc[:,cols-1]
test_features = val_split_test.iloc[:,0:cols-1]
test_target = val_split_test.iloc[:, cols-1]


test_fold = np.zeros((np.shape(data_val_proc)[0],1))  # Referenced from lab 3
test_fold[0:np.shape(val_split_train)[0]] = -1  #Referenced from lab 3
predef_split = PredefinedSplit(test_fold)

n_estimators = np.linspace(start = 20, stop = 200, num = 5, dtype = np.int64)   # Creating the grids for hyperparameters
max_features = np.linspace(start = 1, stop = 5, num =  5, dtype = np.int64)
max_samples = np.linspace(start = 50, stop = 920, num = 5, dtype = np.int64)
params_grid = dict(n_estimators = n_estimators, max_features = max_features, max_samples = max_samples)

grid_regression = GridSearchCV(RandomForestRegressor(), param_grid = params_grid, cv=predef_split, scoring='neg_mean_squared_error')
grid_regression.fit(data_val_features, data_val_target)

print(grid_regression.best_params_)

# Defining the model with best parameters 
regressor = RandomForestRegressor(n_estimators=grid_regression.best_params_["n_estimators"], max_features = grid_regression.best_params_["max_features"], max_samples=grid_regression.best_params_["max_samples"])
regressor.fit(train_features, train_target)

{'max_features': 4, 'max_samples': 920, 'n_estimators': 110}


RandomForestRegressor(max_features=4, max_samples=920, n_estimators=110)

### Question 2.d: train a new model over the whole training data and report the prediction over the test set (4 marks)


Now that we have identified the best paramaters of the regression model, we use these parameters to train a new model over the whole training data (`data_training` plus `data_val`). We apply this model to the test set and report the performance.

* Create a new preprocessing pipeline for taking care of the missing values and standardisation over the whole training data (**1 mark**).

* Apply the created preprocessing pipeline to the test data (**1 mark**).

* Fit a random forest regression model to the training data using the best parameters found in Question 2.c (**1 mark**).

* Compute the RMSE over the test data and report the result (**1 mark**).

#### Question 2.d Answer

In [519]:
from sklearn.metrics import mean_squared_error

training_data_new = pd.concat([data_training_unproc, data_val_unproc])
training_data_new_features = training_data_new.iloc[:,1:cols]
training_data_new_target = training_data_new.iloc[:,0]

model_pipeline = Pipeline(steps = [('imputation', SimpleImputer(missing_values = -200, strategy = 'mean')),
                                   ('standardisation', StandardScaler())])

pre_processor = ColumnTransformer([('preprocess_features', model_pipeline, training_data_new.iloc[:,1:cols].columns)], remainder = 'passthrough') 

pre_processor.fit(training_data_new_features)
training_data_features_transformed = pre_processor.transform(training_data_new_features)

data_test_features = data_test_unproc.iloc[:,1:cols]
data_test_target = data_test_unproc.iloc[:,0]

data_test_features_transformed = pre_processor.transform(data_test_features)

regressor.fit(training_data_new_features, training_data_new_target)

data_test_predictions = regressor.predict(data_test_features)
error_rmse = np.sqrt(mean_squared_error(data_test_target, data_test_predictions))
print("RMSE over the test data:", error_rmse)

RMSE over the test data: 5.812704134776316
